In [ ]:
# default_exp data.cifar

In [ ]:
# hide
%load_ext lab_black
%load_ext autoreload
%autoreload 2

# CIFAR10
> CIFAR10 DataModule

In [ ]:
# export

import pytorch_lightning as pl
import torchvision.transforms as T
import albumentations as A

from torchvision import datasets
from torch.utils.data import DataLoader, random_split
from albumentations.core.composition import BaseCompose
from albumentations.core.transforms_interface import BasicTransform

from typing import Callable, Optional

In [ ]:
# export
class CIFAR10(datasets.CIFAR10):
    """CIFAR10 Dataset wrapper for the torchvision CIFAR10 module, Supports albumentations
    as well as the torchvision transforms

    Args:
        data_dir (:obj:`str`): the root folder for the dataset
        train (bool): either you need the train dataset or the test dataset
        transform (`Callable`): the transforms to perform on the images
        target_transform (`Callable`): the transform to perform on the labels
    """

    mean = (0.4914, 0.4822, 0.4465)
    std = (0.2023, 0.1994, 0.2010)
    num_classes = 10
    dims = (3, 32, 32)

    def __init__(
        self,
        data_dir: str = "~/data/cifar",
        train: bool = True,
        download=True,
        transform: Optional[Callable] = None,
        target_transform: Optional[Callable] = None,
    ):
        super(CIFAR10, self).__init__(
            data_dir,
            train=train,
            download=download,
            transform=transform,
            target_transform=target_transform,
        )

    def __getitem__(self, index):
        image, label = self.data[index], self.targets[index]

        if self.transform is not None:
            if isinstance(self.transform, (BaseCompose, BasicTransform)):
                transformed = self.transform(image=image)
                image = transformed["image"]
            else:
                image = self.transform(image)

        if self.target_transform is not None:
            label = self.target_transform(label)

        return image, label

In [ ]:
# export
class CIFAR10DataModule(pl.LightningDataModule):
    """The CIFAR10 DataModule for Pytorch Lightning
    
    Args:
        data_dir (:obj:`str`): the root folder to store the downloaded dataset
        transform: transforms to apply on the dataset
        test_transform: transforms to apply on the test dataset,
            default will normalize the dataset
    """
    def __init__(
        self,
        data_dir: str = "~/data/cifar",
        transform=T.Compose([T.ToTensor(), T.Normalize(CIFAR10.mean, CIFAR10.std)]),
        test_transform=T.Compose(
            [T.ToTensor(), T.Normalize(CIFAR10.mean, CIFAR10.std)]
        ),
    ):
        super().__init__()
        self.data_dir = data_dir
        self.transform = transform
        self.test_transform = test_transform
        self.mean = CIFAR10.mean
        self.std = CIFAR10.std
        self.num_classes = CIFAR10.num_classes
        self.dims = CIFAR10.dims

    def prepare_data(self):
        # download
        CIFAR10(self.data_dir, train=True, download=True)
        CIFAR10(self.data_dir, train=False, download=True)

    def setup(self, stage=None):

        # Assign train/val datasets for use in dataloaders
        if stage == "fit" or stage is None:
            cifar_full = CIFAR10(self.data_dir, train=True, transform=self.transform)
            self.cifar_train, self.cifar_val = random_split(cifar_full, [45000, 5000])

        # Assign test dataset for use in dataloader(s)
        if stage == "test" or stage is None:
            self.cifar_test = CIFAR10(
                self.data_dir, train=False, transform=self.test_transform
            )

    def train_dataloader(self):
        return DataLoader(
            self.cifar_train, batch_size=32, num_workers=2, pin_memory=True
        )

    def val_dataloader(self):
        return DataLoader(self.cifar_val, batch_size=32, num_workers=2, pin_memory=True)

    def test_dataloader(self):
        return DataLoader(
            self.cifar_test, batch_size=32, num_workers=2, pin_memory=True
        )

In [ ]:
from nbdev.export import notebook2script

notebook2script()

Converted 00_model.dummy.ipynb.
Converted 01_data.cifar.ipynb.
Converted index.ipynb.
Converted template_nb.ipynb.
